In [1]:
#import necessary libraries
import pandas as pd

In [7]:
#Read csv
data = pd.read_csv (r'test.csv')

In [8]:
df = pd.DataFrame(data, columns= ['id','raw_address'])
print(df)

          id                                        raw_address
0          0              s. par 53 sidanegara 4 cilacap tengah
1          1              angg per, baloi indah kel. lubuk baja
2          2                              asma laun, mand imog,
3          3      ud agung rej, raya nga sri wedari karanganyar
4          4                         cut mutia, 35 baiturrahman
...      ...                                                ...
49995  49995                    toko mbak farid semboro semboro
49996  49996     vie - tk. ridho kids, vete 3 cari, 16720 ciawi
49997  49997                mart dan roti bakar malabar, nasio,
49998  49998  graha indah pamulang jl. mujair raya bambu apu...
49999  49999                                               adi,

[50000 rows x 2 columns]


In [13]:
#Shape
df.shape

(50000, 2)

In [14]:
#Call all the raw addresses
df.raw_address

0                    s. par 53 sidanegara 4 cilacap tengah
1                    angg per, baloi indah kel. lubuk baja
2                                    asma laun, mand imog,
3            ud agung rej, raya nga sri wedari karanganyar
4                               cut mutia, 35 baiturrahman
                               ...                        
49995                      toko mbak farid semboro semboro
49996       vie - tk. ridho kids, vete 3 cari, 16720 ciawi
49997                  mart dan roti bakar malabar, nasio,
49998    graha indah pamulang jl. mujair raya bambu apu...
49999                                                 adi,
Name: raw_address, Length: 50000, dtype: object

In [17]:
#Get a rows of addresses
df.loc[2:6]

,id,raw_address
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"
5,5,pem dos dapur ala perum gar no a 12 suka jaya ...
6,6,"tb. mara, cisayong"
